# BC COVID-19 Data by Health Authority

1. [Feature Server JSON](https://services1.arcgis.com/xeMpV7tU1t4KD3Ei/arcgis/rest/services/COVID19_Cases_by_BC_Health_Authority/FeatureServer/0?f=pjson) `id 1,2,3,4,5`

2. [Case Details](http://www.bccdc.ca/health-info/diseases-conditions/covid-19/data)

---

In [5]:
import numpy as np
import pandas as pd
import geopandas as gpd
import requests
from shapely.geometry import shape

In [6]:
r = requests.get("https://services1.arcgis.com/xeMpV7tU1t4KD3Ei/arcgis/rest/services/COVID19_Cases_by_BC_Health_Authority/FeatureServer/0/query?where=&objectIds=1%2C2%2C3%2C4%2C5&time=&geometry=&geometryType=esriGeometryPolygon&inSR=&spatialRel=esriSpatialRelIntersects&resultType=none&distance=0.0&units=esriSRUnit_Meter&returnGeodetic=false&outFields=&returnGeometry=true&returnCentroid=false&featureEncoding=esriDefault&multipatchOption=xyFootprint&maxAllowableOffset=&geometryPrecision=&outSR=&datumTransformation=&applyVCSProjection=false&returnIdsOnly=false&returnUniqueIdsOnly=false&returnCountOnly=false&returnExtentOnly=false&returnQueryGeometry=true&returnDistinctValues=false&cacheHint=false&orderByFields=&groupByFieldsForStatistics=&outStatistics=&having=&resultOffset=&resultRecordCount=&returnZ=false&returnM=false&returnExceededLimitFeatures=true&quantizationParameters=&sqlFormat=none&f=pgeojson&token=")
r.raise_for_status()

data = r.json()

In [7]:
gdf = gpd.GeoDataFrame.from_features(data["features"])
print(gdf.head())

             HA_Name                                           geometry
0           Interior  POLYGON ((-120.45314 52.99326, -120.42102 52.9...
1             Fraser  MULTIPOLYGON (((-122.93676 49.31128, -122.9365...
2  Vancouver Coastal  MULTIPOLYGON (((-123.10903 49.28623, -123.0231...
3   Vancouver Island  MULTIPOLYGON (((-126.66725 51.19230, -126.6705...
4           Northern  MULTIPOLYGON (((-129.76908 55.45115, -129.7657...


In [8]:
with open('../data/bc-health-authorities.geojson', 'w') as f:
    f.write(gdf.to_json())

---

2. **Case Details**

In [9]:
pd.read_csv('../data/BCCDC_COVID19_Dashboard_Case_Details.csv')

FileNotFoundError: [Errno 2] File b'../data/BCCDC_COVID19_Dashboard_Case_Details.csv' does not exist: b'../data/BCCDC_COVID19_Dashboard_Case_Details.csv'

---
# BC COVID Data for LTCs

1. [long term facilities](https://www.seniorsadvocatebc.ca/quickfacts/search/%20)
2. [outbreaks](http://www.phsa.ca/current-outbreaks)

---

1. **Long term facilities**

In [10]:
from bs4 import BeautifulSoup

In [11]:
# user defined exception errors
class Error(Exception):
    """Base class for other exceptions"""
    pass

class InvalidHome(Error):
    """Raised when return is not a home name or there is a duplicate"""
    pass

class InvalidAddress(Error):
    """Raised when return is not a long-term care home address"""
    pass

In [ ]:
address = []
home = []
links = []
    
for i in range(1,31):
    re = requests.get('https://www.seniorsadvocatebc.ca/quickfacts/search/%20/'+ str(i))
    s = BeautifulSoup(re.text,'html5lib')

    containers = s.find_all("div")

    for each in containers:
        try:
             # home address
            if 'Address:' not in each.find('p').getText():
                raise InvalidAddress
            else:
                address.append(each.find('p').getText())

            # home name
            if each.find('a').getText() in ['Home', '\n\t\t\t\t\t\t\t«\n\t\t\t\t\t\t', 
                                            'Data Sources', 'Visit us on Facebook', 
                                            'Email the OSA', '']:
                raise InvalidHome
            elif each.find('a').getText() in home:
                # remove duplicates (if home name already exists)
                raise InvalidHome
            else:
                home.append(each.find('a').getText())

            # home url
            if each.find('a').get('href') in links:
                # remove duplicates (if url already exists)
                raise InvalidHome
            else:
                links.append(each.find('a').get('href'))

        except AttributeError:
            pass
        except InvalidHome:
            pass
        except InvalidAddress:
            pass

In [ ]:
ltc = pd.DataFrame({'Facility':home, 'Link':links, 'Street address':address})

In [ ]:
# clean and separate address info
ltc['Street address'] = ltc['Street address'].str.replace('\n\t\t\t\t',' ')
ltc['city/postal'] = ltc['Street address'].str.extract(r'City/postal:(.*) Phone:')
ltc['phone'] = ltc['Street address'].str.extract(r'Phone:(.*)\t\t\t')
ltc['Street address'] = ltc['Street address'].str.extract(r'Address:(.*) City/postal:')

In [ ]:
ltc

### **Web Crawler for Home Details**

Note: did not include  'Disease outbreak or occurrence' since 21 are missing. would need a try except statement.

In [ ]:
reques = requests.get(links[0])
soup = BeautifulSoup(reques.text, 'html5lib')

alltext = []
for t in soup.find_all('td'):
    alltext.append(t.getText())
    
    
keep = ['\n\t\t\t\t\t\t\n\t\t\t\t\t', 'Health authority', 'Accreditation status', 'Councils', 'Private beds (not publicly funded)', 'Publicly funded beds (short- and long-term)',
       'Total beds', 'Private rooms', 'Semi-private rooms', 'Multi-person rooms', 'Total direct care hours (hours per resident per day)',
       'Number of licensing complaints', 'Number substantiated licensing complaints',
       'Number of Inspections', 'Number of licensing infractions found', 'Hygiene & communicable disease control', 'Average age of population']


# initialize empty dict
details = {}

def curate(x, keep):
    '''param x: is a list of text scraped from LTC tables
        param keep: is a list of key words to wait for before selecting text'''
    index = -1

    if not bool(details):
        # if dictionary is empty, create the keys and values
        for i in x:
            index += 1
            if i in keep:
                details[i] = [x[index+1]]
        
    else:
        # if the dictionary already has entries, append new values
        for i in x:
            index += 1
            if i in keep:
                details[i].append(x[index+1])
            
    return details

***Warning:*** slow script below

In [ ]:
for l, i in zip(links, range(1, len(links)+1)):
    print('making beautiful soup - {} of {}'.format(i, len(links)))
    re = requests.get(l)
    s = BeautifulSoup(re.text,'html5lib')
    complete = []

    for t in s.find_all('td'):
        # SLOWEST STEP
        complete.append(t.getText())

    curate(complete, keep)

In [ ]:
# summary of contents
for k in details.keys():
    print(k, len(details[k]))

df = pd.DataFrame(details)

# clean facility name scrape
df.rename(columns = {'\n\t\t\t\t\t\t\n\t\t\t\t\t' : 'Facility'}, inplace = True)
df.Facility = df.Facility.str.replace('\n\t\t\t\t\t\t', '')
df.Facility = df.Facility.str.replace(' Quick Facts(.*)\n\t\t\t\t\t', '')
df

In [ ]:
ltc = pd.merge(ltc, df, how = 'outer', on = 'Facility')
ltc.tail()

2. **Outbreaks**

From: [Nora Loreto](https://docs.google.com/spreadsheets/d/1M_RzojK0vwF9nAozI7aoyLpPU8EA1JEqO6rq0g1iebU/edit#gid=0)

*Up to date as of 23h July 21*

---

In [ ]:
cases = pd.read_csv('../data/Deaths in Residential Care in Canada by facility - Institutions.csv',  skiprows=[0,1])


In [ ]:
# BC facilities
BC_cases = cases.loc[cases['Prov.'] == 'BC']
BC_cases.info()

In [ ]:
# drop unneccessary columns
dr = ['Owners', 'Union', 'Unnamed: 9', 'Unnamed: 10']

BC_cases.drop(columns = dr, inplace = True)

### Clean and Merge LTC Details with COVID-19 Outbreak Information

---

In [ ]:
merge = pd.merge(BC_cases, ltc, on = 'Facility', how = 'inner')

In [ ]:
list(set(merge.Facility.unique()) - set(BC_cases.Facility.unique()))

In [ ]:
list(set(BC_cases.Facility.unique()) - set(merge.Facility.unique()))

**Unaccounted for homes in initial merge:**

Name in BC Cases | In LTC list
-----------------|-------------
Holy Family Hospital long term care unit | Holy Family Hospital
Haro Park | Haro Park Centre
Lions Gate Hospital (Unit 6 East) | [Evergreen House - Lions Gate Hospital](https://www.google.com/search?client=safari&rls=en&q=lions+gate+hospital+unit+6+evergreen+house&ie=UTF-8&oe=UTF-8)
Four LTC homes in Vancouver - July 6 | `unknown`
Amica Edgemont Village | `missing - senior living`
Ridge Meadows Hospital (Rehab and Unit 3W) | [Baillie House](https://www.fraserhealth.ca/Service-Directory/Service-At-Location/5/7/emergency---ridge-meadows-hospital#.Xx8Rpy0ZNQI)
Swedish Assisted Living | `missing - assisted living`
Worthington Longterm Care Facility | [Cottage and Worthington Pavilions - MSA Hospital](https://www.fraserhealth.ca/Service-Directory/Locations/Abbotsford/cottage-worthington-pavilion#.Xx8PRy0ZNQI)
Residence at Clayton Heights | The Residence at Clayton Heights
South Granville Park Lodge | `missing`
Shaughnessy Care Centre | `missing`
Valhaven Home | Valhaven Rest Home
Mission Institution* | `correctional facility`

***Correct nomenclature to match those in LTC scrape:***

In [ ]:
BC_cases.Facility.replace({'Holy Family Hospital long term care unit' : 'Holy Family Hospital',
                           'Haro Park' : 'Haro Park Centre',
                           'Lions Gate Hospital (Unit 6 East)' : 'Evergreen House - Lions Gate Hospital',
                           'Worthington Longterm Care Facility' : 'Cottage and Worthington Pavilions - MSA Hospital',
                           'Residence at Clayton Heights' : 'The Residence at Clayton Heights',
                           'Ridge Meadows Hospital (Rehab and Unit 3W)' : 'Baillie House',
                           'Valhaven Home':'Valhaven Rest Home'}, inplace = True)

In [ ]:
merge = pd.merge(BC_cases, ltc, on = 'Facility', how = 'right')
list(set(BC_cases.Facility.unique()) - set(merge.Facility.unique()))

#### As done in Ontario analysis, add binary columns for councils and accreditation

In [1112]:
def residents(x):
    if 'Resident' in x:
        return 'yes'
    else:
        return 'no'
    
    
def family(x):
    if 'Family' in x:
        return 'yes'
    else:
        return 'no'

def accred(x):
    if x == 'Not Accredited':
        return 'no'
    elif x == 'Accredited':
        return 'yes'
    else:
        return 'unknown'
    
merge['residents_council'] = merge.Councils.map(lambda x: residents(x))
merge['family_council'] = merge.Councils.map(lambda x: family(x))
merge['accreditation'] = merge['Accreditation status'].map(lambda x: accred(x))

#### Add outbreak status column

In [1104]:
def outbreak(x):
    try:
        if int(x) != np.nan:
            return "yes"
    except ValueError:
        return "no"
    
merge['outbreak_status'] = merge['Number of dead'].map(lambda x: outbreak(x))

Manually adjust outbreak status from webscrapes (supplementary portion at bottom of this .ipynb)

In [1514]:
# July 25th Scrape from Northern
merge.outbreak_status.loc[merge.Facility == 'Terraceview Lodge'] = 'yes'

# Aug 4th Scrape from Fraser
merge.outbreak_status.loc[merge.Facility == 'Dania Home'] = 'yes'

# Aug 10th Scrape from Fraser
merge.outbreak_status.loc[merge.Facility == 'Maple Ridge Seniors Village'] = 'yes'
# merge.outbreak_status.loc[merge.Facility == 'Derby Manor'] = 'yes' - NOT FOUND
merge.outbreak_status.loc[merge.Facility == 'George Derby Centre'] = 'yes'
# merge.outbreak_status.loc[merge.Facility == 'New Vista Care Society'] = 'yes' - NOT FOUND

A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy [indexing.py:671]


In [1512]:
merge.query('Facility.str.contains("Derby Manor") or Facility.str.contains("New Vista Care Society")', engine = 'python')

,Facility,Location,Prov.,Number of dead,Number of beds (appx),%,Public or private,Link,Street address,city/postal,...,Number of licensing complaints,Number substantiated licensing complaints,Number of Inspections,Number of licensing infractions found,Hygiene & communicable disease control,Average age of population,outbreak_status,residents_council,family_council,accreditation


In [1105]:
merge.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 297 entries, 0 to 296
Data columns (total 28 columns):
 #   Column                                                Non-Null Count  Dtype  
---  ------                                                --------------  -----  
 0   Facility                                              297 non-null    object 
 1   Location                                              15 non-null     object 
 2   Prov.                                                 15 non-null     object 
 3   Number of dead                                        15 non-null     float64
 4   Number of beds (appx)                                 3 non-null      float64
 5   %                                                     11 non-null     object 
 6   Public or private                                     11 non-null     object 
 7   Link                                                  297 non-null    object 
 8   Street address                                        297 no

### Caveats:

- Nora Loreto's information may not be complete (e.g. Terraceview Lodge from VCH scrape)
- Nora Loreto reports public or private status of home, which is not found on BC government websites (as of yet) and does not reflect private or publicly funded beds

## Home Type Scrape 

From: [FOI Request (2018)](http://docs.openinfo.gov.bc.ca/Response_Package_HTH-2018-83925.pdf)

In [1161]:
from PyPDF2 import PdfFileReader
import io
import re

hometype = requests.get('http://docs.openinfo.gov.bc.ca/Response_Package_HTH-2018-83925.pdf')

with io.BytesIO(hometype.content) as f:
            pdf = PdfFileReader(f)
            information = pdf.getDocumentInfo()
            pages = pdf.getNumPages()
            print(pdf.getPageLayout())
            for i in range(2, 9):
                page = pdf.getPage(i)
                print(page)
                page_content = page.getContents()           
                print(page_content)

None
{'/Type': '/Page', '/MediaBox': [0, 0, 792, 612], '/Parent': IndirectObject(3, 0), '/Resources': {'/XObject': {'/Im3': IndirectObject(21, 0)}, '/ProcSet': ['/PDF', '/ImageI']}, '/Contents': IndirectObject(19, 0)}
{}
{'/Type': '/Page', '/MediaBox': [0, 0, 792, 612], '/Parent': IndirectObject(3, 0), '/Resources': {'/XObject': {'/Im4': IndirectObject(28, 0)}, '/ProcSet': ['/PDF', '/ImageI']}, '/Contents': IndirectObject(26, 0)}
{}
{'/Type': '/Page', '/MediaBox': [0, 0, 792, 612], '/Parent': IndirectObject(3, 0), '/Resources': {'/XObject': {'/Im5': IndirectObject(35, 0)}, '/ProcSet': ['/PDF', '/ImageI']}, '/Contents': IndirectObject(33, 0)}
{}
{'/Type': '/Page', '/MediaBox': [0, 0, 792, 612], '/Parent': IndirectObject(3, 0), '/Resources': {'/XObject': {'/Im6': IndirectObject(42, 0)}, '/ProcSet': ['/PDF', '/ImageI']}, '/Contents': IndirectObject(40, 0)}
{}
{'/Type': '/Page', '/MediaBox': [0, 0, 792, 612], '/Parent': IndirectObject(3, 0), '/Resources': {'/XObject': {'/Im7': IndirectObje

<mark> **Since PDF charts are images and can not be scraped, home type was recorded manually in reference to the released document.**</mark>


In [1172]:
types = pd.read_csv('/Users/kt/Documents/covid-research/data/bc_home_types.csv')

In [1173]:
types.head()

,Health authority,city/postal,Facility,owner_type,comments,home_type
0,Fraser Health,Abbotsford V2S0B3,Cottage and Worthington Pavilions - MSA Hospital,HA,NaN,health_authority
1,Fraser Health,Abbotsford V2S1K1,Menno Hospital,PNP,NaN,non-profit
2,Fraser Health,Abbotsford V2S1K8,The Mayfair,PNP,NaN,non-profit
3,Fraser Health,Abbotsford V2S1N2,Menno Home,PNP,NaN,non-profit
4,Fraser Health,Abbotsford V2S2Z8,Maplewood House,PNP,NaN,non-profit


In [1515]:
merge.to_csv('../data/bc_ltc.csv')

---

---
---

# ***FIN***

---
---
---

# Outbreak Data Web Scrape

2. **Outbreaks**

---

***Northern***

In [3]:
re = requests.get('https://www.northernhealth.ca/health-topics/current-outbreaks')
s = BeautifulSoup(re.text,'html5lib')
table =s.find_all('td')

NameError: name 'BeautifulSoup' is not defined

In [1175]:
d = ['city', 'facility', 'outbreaktype', 'datedeclared']
outbreak = {}
for each,i in zip(table,d):
    outbreak[i] = each.text.strip()

In [1176]:
northern = pd.DataFrame(outbreak, index = [0])
northern['health_authority'] = "northern"

In [1177]:
northern.datedeclared = northern.datedeclared.str.replace('\n\n\t\t\t', '-')

In [1178]:
northern

,city,facility,outbreaktype,datedeclared,health_authority
0,None to date,,,,northern


***Interior Health***

In [1179]:
ih = requests.get('https://www.interiorhealth.ca/YourEnvironment/CommunicableDiseaseControl/Outbreaks/Pages/default.aspx')
ih = BeautifulSoup(ih.text,'html5lib')
outbreaks = ih.find_all('table')[7]

In [1180]:
outbreaks.find('td').text

'There are no items to show in this view of the "Outbreaks" list.'

***Vancouver Island***

In [1181]:
vi = requests.get('https://www.healthspace.ca/Clients/VIHA/VIHA_Website.nsf/Outbreak')
vi = BeautifulSoup(vi.text,'html5lib')
outbreaks = vi.find('td')

In [1182]:
outbreaks.getText()

' Currently, there are no reported outbreaks in VIHA hospitals or long term care facilities.'

***Vancouver Coastal Health***

In [1183]:
from PyPDF2 import PdfFileReader
import io
import re

In [1184]:
vch = requests.get('http://www.vch.ca/Documents/facility-outbreak-bulletin.pdf')

In [1185]:
with io.BytesIO(vch.content) as f:
            pdf = PdfFileReader(f)
            information = pdf.getDocumentInfo()
            pages = pdf.getNumPages()
            for i in range(0, pages):
                page = pdf.getPage(i)
                page_content = page.extractText()           
                print(page_content)

Facility Outbreak Bulletin
This bulletin lists ongoing and recently ended outbreaks in licensed long-term and acute care
facilities throughout Vancouver Coastal Health, as of 01:00 PM, 06-Aug-2020
DISEASE
LOCATION
RESTRICTIONS
IMPOSED
RESTRICTIONS
LIFTED
FACILITY
COVID-19
9020 Bridgeport Road, Richmond
05-Aug-20
Richmond Lions Manor - Bridgeport, 2nd Floor
COVID-19
803 West 12th Avenue, Vancouver
06-Aug-20
Joseph & Rosalie Segal Family Health Centre, 8th 
Floor
COVID-19
1081 Burrard Street, Vancouver
16-Jul-20
28-Jul-20
St. Paul's Hospital, NICU
COVID-19
7801 Argyle St, Vancouver
09-Jun-20
Holy Family Hospital, LTCF (Rehabilitation Unit 
declared over)
Page 1 of 1
Red text denotes updates from previously issued bulletin
Grey text indicates that restrictions have been lifted
Restriction Imposed: 
Restrictions Lifted: 
               Date which outbreak measures were introduced
            Date which outbreak measures were discontinued



----

In [1469]:
spl = page_content.split('\n')

In [1470]:
spldf = pd.Series(spl)

In [1471]:
spldf.iloc[10:29]

10                                             COVID-19
11                       9020 Bridgeport Road, Richmond
12                                            05-Aug-20
13         Richmond Lions Manor - Bridgeport, 2nd Floor
14                                             COVID-19
15                      803 West 12th Avenue, Vancouver
16                                            06-Aug-20
17    Joseph & Rosalie Segal Family Health Centre, 8th 
18                                                Floor
19                                             COVID-19
20                       1081 Burrard Street, Vancouver
21                                            16-Jul-20
22                                            28-Jul-20
23                            St. Paul's Hospital, NICU
24                                             COVID-19
25                            7801 Argyle St, Vancouver
26                                            09-Jun-20
27     Holy Family Hospital, LTCF (Rehabilitatio

In [1484]:
vch_outbreaks = pd.DataFrame({'info' : spldf.iloc[11:29]})

In [1483]:
vch_outbreaks.groupby(vch_outbreaks.index // 5).agg('\n'.join)['info'].str.split('\n',expand=True)

,0,1,2,3,4
2,"9020 Bridgeport Road, Richmond",05-Aug-20,"Richmond Lions Manor - Bridgeport, 2nd Floor",COVID-19,None
3,"803 West 12th Avenue, Vancouver",06-Aug-20,"Joseph & Rosalie Segal Family Health Centre, 8th",Floor,COVID-19
4,"1081 Burrard Street, Vancouver",16-Jul-20,28-Jul-20,"St. Paul's Hospital, NICU",COVID-19
5,"7801 Argyle St, Vancouver",09-Jun-20,"Holy Family Hospital, LTCF (Rehabilitation Unit",declared over),None


In [1473]:
vch_outbreaks.rename(columns = {'address', 'restrictionsimposed', 'facility', 'outbreaktype'} inplace = True)

In [4]:
vch_outbreaks

NameError: name 'vch_outbreaks' is not defined

In [622]:
vch_pivot = vch_outbreaks.pivot_table(values='info',
                                     columns='details', 
                                     aggfunc=lambda x: '\n'.join(x))

In [623]:
vch_pivot

details,address,facility,outbreaktype,restrictionsimposed,restrictionslifted,status
info,"1081 Burrard Street, Vancouver\n7801 Argyle St...","St. Paul's Hospital, NICU\nHoly Family Hospita...",COVID-19\nCOVID-19\nCOVID-19\nCOVID-19,16-Jul-20\n09-Jun-20\n12-Apr-20\n30-Mar-20,13-Jun-20\n05-Jun-20,declared over)


In [640]:
def vch_split(x):
    for r in x.splitlines():
        return r
    
vch_pivot.address.apply(lambda x: vch_split(x))

info    1081 Burrard Street, Vancouver
Name: address, dtype: object

In [630]:
vch_pivot['health_authority'] = 'vancouver coastal health'

***Fraser***

In [1197]:
f = requests.get(r'https://www.fraserhealth.ca/patients-and-visitors/current-outbreaks#.XzGwMS0ZNQI')
fraser = BeautifulSoup(f.text,'html5lib')
table = fraser.find_all('td')

In [1446]:
newlist = []

for td in table:
        try:
            if td.find('h5').getText() == '\xa0':
                # if no info is provided, it will mess up indexing
                pass
            elif td.find('h5').getText() == '':
                # if no info is provided, it will mess up indexing
                pass
            else:    
                newlist.append(td.find('h5').getText())
        except AttributeError:
            pass
        
            
        
df = pd.DataFrame(newlist)

In [1447]:
newlist

['July 31, 2020',
 'Dania Home',
 'Residential',
 'COVID-19',
 'August 4, 2020',
 'Maple Ridge Seniors Village',
 'Residential',
 'COVID-19',
 'August 6, 2020',
 'Derby Manor',
 'Residential',
 'COVID-19',
 'August 7, 2020',
 'George Derby Centre',
 'Residential',
 'COVID-19',
 'August 8, 2020',
 'New Vista Care Society',
 'Residential',
 'COVID-19']

In [1464]:
fraser = df.groupby(df.index // 4).agg('\n'.join)[0].str.split('\n',expand=True)

In [1465]:
fraser.rename(columns = {0:'datedeclared', 1:'facility', 2:'facilitytype', 3:'outbreaktype'}, inplace = True)

In [1466]:
fraser['health_authority'] = 'fraser'

In [1467]:
fraser

,datedeclared,facility,facilitytype,outbreaktype,health_authority
0,"July 31, 2020",Dania Home,Residential,COVID-19,fraser
1,"August 4, 2020",Maple Ridge Seniors Village,Residential,COVID-19,fraser
2,"August 6, 2020",Derby Manor,Residential,COVID-19,fraser
3,"August 7, 2020",George Derby Centre,Residential,COVID-19,fraser
4,"August 8, 2020",New Vista Care Society,Residential,COVID-19,fraser


### Merge facilities

In [649]:
# merge1 = pd.merge(northern, fraser, how = 'outer')

In [1516]:
merge1

,city,facility,outbreaktype,datedeclared,health_authority,facilitytype,unit
0,Terrace,Terraceview Lodge - Lakelse Unit,Respiratory Illness,"April 9, 2020-Declared over: April 13, 2020",northern,NaN,NaN
1,NaN,Mission Memorial Hospital,COVID-19,"June 16, 2020",fraser,Hospital,


In [650]:
# merge2 = pd.merge(merge1, vch_pivot, how = 'outer')

In [651]:
merge2

,city,facility,outbreaktype,datedeclared,health_authority,facilitytype,unit,address,restrictionsimposed,restrictionslifted,status
0,Terrace,Terraceview Lodge - Lakelse Unit,Respiratory Illness,"April 9, 2020-Declared over: April 13, 2020",northern,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,Mission Memorial Hospital,COVID-19,"June 16, 2020",fraser,Hospital,,NaN,NaN,NaN,NaN
2,NaN,"St. Paul's Hospital, NICU\nHoly Family Hospita...",COVID-19\nCOVID-19\nCOVID-19\nCOVID-19,NaN,vancouver coastal health,NaN,NaN,"1081 Burrard Street, Vancouver\n7801 Argyle St...",16-Jul-20\n09-Jun-20\n12-Apr-20\n30-Mar-20,13-Jun-20\n05-Jun-20,declared over)


In [1468]:
# update new scrapes
aug4 = pd.merge(merge2, fraser, how = 'outer')

# aug 10
aug10 = pd.merge(merge2, fraser, how = 'outer')
aug10

,city,facility,outbreaktype,datedeclared,health_authority,facilitytype,unit,address,restrictionsimposed,restrictionslifted,status
0,Terrace,Terraceview Lodge - Lakelse Unit,Respiratory Illness,"April 9, 2020-Declared over: April 13, 2020",northern,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,Mission Memorial Hospital,COVID-19,"June 16, 2020",fraser,Hospital,,NaN,NaN,NaN,NaN
2,NaN,"St. Paul's Hospital, NICU\nHoly Family Hospita...",COVID-19\nCOVID-19\nCOVID-19\nCOVID-19,NaN,vancouver coastal health,NaN,NaN,"1081 Burrard Street, Vancouver\n7801 Argyle St...",16-Jul-20\n09-Jun-20\n12-Apr-20\n30-Mar-20,13-Jun-20\n05-Jun-20,declared over)
3,NaN,Dania Home,COVID-19,"July 31, 2020",fraser,Residential,NaN,NaN,NaN,NaN,NaN
4,NaN,Maple Ridge Seniors Village,COVID-19,"August 4, 2020",fraser,Residential,NaN,NaN,NaN,NaN,NaN
5,NaN,Derby Manor,COVID-19,"August 6, 2020",fraser,Residential,NaN,NaN,NaN,NaN,NaN
6,NaN,George Derby Centre,COVID-19,"August 7, 2020",fraser,Residential,NaN,NaN,NaN,NaN,NaN
7,NaN,New Vista Care Society,COVID-19,"August 8, 2020",fraser,Residential,NaN,NaN,NaN,NaN,NaN


# Caveats:

1. VCH outbreak status is colour coded and requires custom scripts for each new outbreak
2. All only report active outbreaks (no information on previous outbreaks)
3. VCH also reports hospital outbreaks - not just LTCs